In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, cohen_kappa_score
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv('dataset.csv', index_col=0)

In [3]:
df

,full_text,grade,num_para,num_word_div_para,MTLD,prompt,Overall,Cohesion,Syntax,Vocabulary,...,avg_modals_normalized,avg_preps_normalized,avg_punct_normalized,word_count,char_count,unique_word_count,tf_ratio_word_bigram,tf_ratio_word_trigram,tf_ratio_pos_bigram,tf_ratio_pos_trigram
0,Imagine if you could prove other people that y...,8,4,107.750000,28.626289,Benefits of a problem,4.0,3.5,4.0,3.5,...,0.049247,0.063143,0.066954,435,2317,109,1254.893461,3132.691677,1.537062,5.790256
1,I think that students would benefit from learn...,12,4,65.250000,66.021538,Distance learning,3.5,3.5,3.5,3.0,...,0.045367,0.059434,0.093017,267,1387,130,1546.321274,5828.006549,1.738246,7.887585
2,When a problem is a change you have to let it ...,8,6,88.833333,33.216069,Benefits of a problem,2.5,2.5,2.5,3.0,...,0.027383,0.060190,0.042822,535,2635,134,452.649281,2066.952169,2.468856,14.168887
3,"Dear, Principal\n\nIf u change the school poli...",8,2,160.000000,49.329499,Grades for extracurricular activities,3.0,3.0,3.5,3.0,...,0.030290,0.064994,0.087846,325,1663,110,1114.334671,4298.734780,2.318318,16.026845
4,The best time in life is when you become yours...,12,8,91.000000,37.516148,Individuality,4.5,4.5,4.5,4.5,...,0.017434,0.048573,0.125828,727,3973,199,497.759267,1654.270131,1.666789,5.695199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6477,Some schools offer distance learning as a opti...,12,7,47.857143,55.862500,Distance learning,3.0,2.5,2.0,3.0,...,0.031348,0.118805,0.095700,336,1886,117,904.847155,2879.215741,1.798651,5.416057
6478,I disagree that attend classes from home can b...,12,1,278.000000,44.787441,Distance learning,3.0,3.0,3.0,2.5,...,0.059070,0.113073,0.075518,280,1545,132,1629.995330,4667.047154,1.949613,9.374072
6479,Introduction in my opinion kids should not sta...,8,8,46.875000,45.142574,Mandatory extracurricular activities,3.0,3.0,3.0,4.0,...,0.040150,0.114526,0.064235,382,2023,155,1540.669913,3945.727271,1.551957,4.610126
6480,"some times, We don't succed in life for reason...",11,16,15.062500,73.611627,Distance learning,3.0,3.5,3.5,3.0,...,0.013785,0.100716,0.121755,239,1406,128,2217.644670,6681.349693,1.850974,6.721453


In [8]:
scores = ['Overall', 'Cohesion', 'Syntax', 'Vocabulary', 'Phraseology', 'Grammar', 'Conventions']

In [9]:
y = df[scores]

In [19]:
scaler = MinMaxScaler()
y = scaler.fit_transform(y)

In [23]:
y = pd.DataFrame(y, columns=scores)

In [10]:
!pip install catboost

In [11]:
X = df.drop(scores, axis=1)

In [12]:
X.columns

Index(['full_text', 'grade', 'num_para', 'num_word_div_para', 'MTLD', 'prompt',
       'avg_word_length', 'var_word_length', 'words_longer_four',
       'words_longer_six', 'words_longer_eight', 'words_longer_ten',
       'words_longer_twelve', 'sent_lengths_10', 'sent_lengths_18',
       'sent_lengths_25', 'sent_avg_length', 'sent_var_length',
       'avg_num_subclauses_norm', 'avg_len_subclauses',
       'avg_modals_normalized', 'avg_preps_normalized', 'avg_punct_normalized',
       'word_count', 'char_count', 'unique_word_count', 'tf_ratio_word_bigram',
       'tf_ratio_word_trigram', 'tf_ratio_pos_bigram', 'tf_ratio_pos_trigram'],
      dtype='object')

In [16]:
X = X.drop(['grade', 'prompt'], axis=1)

In [17]:
from catboost import CatBoostRegressor, Pool

In [24]:
for aspect in scores:
    X_train, X_test, y_train, y_test = train_test_split(X, y[aspect], test_size=0.2, random_state=42)

    train_pool = Pool(X_train, y_train, text_features=['full_text'])
    test_pool = Pool(X_test, text_features=['full_text'])

    model = CatBoostRegressor(iterations=500, depth=6, learning_rate=0.03, verbose=50)
    model.fit(train_pool)

    y_pred = model.predict(test_pool)

    print(aspect)
    print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
    print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred))
    print("R² Score:", r2_score(y_test, y_pred))

    y_pred = y_pred * 4 + 1 # to original values
    y_pred = np.round(y_pred * 2) / 2
    y_test = y_test * 4 + 1
    y_test = np.round(y_test * 2) / 2

    y_pred = (y_pred - 1) * 2 # to integer values
    y_test = (y_test - 1) * 2

    print("QWK Score:", cohen_kappa_score(y_test, y_pred), '\n')

0:	learn: 0.1595454	total: 271ms	remaining: 2m 15s
50:	learn: 0.1317222	total: 13.2s	remaining: 1m 56s
100:	learn: 0.1231624	total: 25.9s	remaining: 1m 42s
150:	learn: 0.1182945	total: 38.5s	remaining: 1m 28s
200:	learn: 0.1147800	total: 51.1s	remaining: 1m 15s
250:	learn: 0.1119060	total: 1m 2s	remaining: 1m 2s
300:	learn: 0.1090418	total: 1m 15s	remaining: 49.7s
350:	learn: 0.1059951	total: 1m 27s	remaining: 37.1s
400:	learn: 0.1030380	total: 1m 40s	remaining: 24.9s
450:	learn: 0.1003676	total: 1m 53s	remaining: 12.3s
499:	learn: 0.0978035	total: 2m 5s	remaining: 0us
Overall
Mean Squared Error: 0.013670501537703397
Mean Absolute Error: 0.09270494889227088
R² Score: 0.42115142949315676
QWK Score: 0.20185071376862163 

0:	learn: 0.1637469	total: 456ms	remaining: 3m 47s
50:	learn: 0.1402371	total: 13.2s	remaining: 1m 56s
100:	learn: 0.1329779	total: 34.9s	remaining: 2m 17s
150:	learn: 0.1289718	total: 47.6s	remaining: 1m 50s
200:	learn: 0.1260155	total: 1m	remaining: 1m 30s
250:	learn: 